

---

# **Chapter 30: Integration & E2E Testing**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Configure and set up the `integration_test` package for end-to-end testing
- Implement complete user flow tests across multiple screens and navigation states
- Execute tests on real devices, emulators, and simulators with proper configuration
- Capture performance metrics and timeline traces during test execution
- Integrate automated tests into CI/CD pipelines (GitHub Actions, Codemagic, Bitrise)
- Handle platform-specific permissions and native features in integration tests
- Implement screenshot capture and visual validation during test runs
- Manage test data and state across complex multi-step workflows

---

## **Prerequisites**

- Completed Chapter 29: Widget Testing (finder strategies, pumpAndSettle)
- Understanding of Flutter app architecture and navigation
- Physical device or emulator/simulator setup for Android/iOS
- Basic understanding of YAML configuration for CI/CD
- Flutter SDK with integration_test package available

---

## **30.1 Introduction to Integration Testing**

Integration tests (also called end-to-end or E2E tests) verify that the complete application works correctly by testing the fully assembled app on real devices or emulators. Unlike unit and widget tests, integration tests use actual platform channels, real network requests, and native device features.

### **The Integration Test Architecture**

```dart
// File: integration_test/app_test.dart
// Integration tests reside in a separate top-level directory

import 'package:flutter/material.dart';
import 'package:flutter_test/flutter_test.dart';
import 'package:integration_test/integration_test.dart';
import 'package:my_app/main.dart' as app;

void main() {
  // Essential: Initialize the integration test binding
  // This connects the test to the native device/emulator
  IntegrationTestWidgetsFlutterBinding.ensureInitialized();
  
  group('End-to-End App Test', () {
    testWidgets('complete login flow', (WidgetTester tester) async {
      // Launch the actual app - not a test widget
      app.main();
      
      // Wait for app to settle (splash screen, initial loading)
      await tester.pumpAndSettle();
      
      // Now interact with the real app
      expect(find.text('Login'), findsOneWidget);
    });
  });
}
```

**Explanation:**

- **`IntegrationTestWidgetsFlutterBinding.ensureInitialized()`**: Critical setup that connects the test framework to the host device. Without this, the test cannot interact with the native platform (gestures, platform channels, etc.).
- **`app.main()`**: Launches the actual application entry point (`lib/main.dart`), not a test mock. This tests the real app code with real dependencies.
- **Real execution**: Unlike widget tests that run in a headless environment, integration tests run on actual devices with real Skia rendering, actual network calls, and real platform plugins.
- **Directory structure**: Integration tests must be in the `integration_test/` folder (parallel to `lib/` and `test/`), not in the `test/` folder.

### **Key Differences from Widget Tests**

```dart
// File: comparison/widget_vs_integration.dart

// WIDGET TEST (test/widget_test.dart)
testWidgets('widget test example', (WidgetTester tester) async {
  // Builds widget in memory, no real app context
  await tester.pumpWidget(MaterialApp(home: MyScreen()));
  
  // Uses mock platform channels (if any)
  // Network calls typically mocked or failed
  
  // Fast execution (milliseconds)
});

// INTEGRATION TEST (integration_test/app_test.dart)
testWidgets('integration test example', (WidgetTester tester) async {
  // Launches full app with all native initialization
  app.main();
  await tester.pumpAndSettle(Duration(seconds: 2));
  
  // Real platform channels active
  // Actual HTTP requests executed
  // Real database operations (SQLite, SharedPreferences)
  // Actual file system access
  
  // Slow execution (seconds to minutes)
});
```

**Explanation:**

- **Scope**: Widget tests isolate a single widget; integration tests exercise the full app stack including native code.
- **Dependencies**: Widget tests mock dependencies; integration tests use real services (production API endpoints, real databases).
- **Performance**: Integration tests are 10-100x slower than widget tests due to real I/O and rendering.
- **Reliability**: Integration tests are flakier due to network latency, animations, and timing issues requiring careful synchronization.

---

## **30.2 Setting Up Integration Tests**

Proper configuration ensures integration tests run consistently across development machines and CI/CD environments.

### **Project Configuration**

```yaml
# File: pubspec.yaml
# Add to dev_dependencies (not regular dependencies)

dev_dependencies:
  flutter_test:
    sdk: flutter
  integration_test:
    sdk: flutter
  # Optional: For screenshot comparison
  flutter_driver:
    sdk: flutter
```

```dart
// File: integration_test/auth_flow_test.dart
// Example: Complete authentication flow test

import 'package:flutter/material.dart';
import 'package:flutter_test/flutter_test.dart';
import 'package:integration_test/integration_test.dart';
import 'package:my_app/main.dart' as app;

void main() {
  IntegrationTestWidgetsFlutterBinding.ensureInitialized();
  
  group('Authentication Flow', () {
    testWidgets('user can login and view dashboard', (WidgetTester tester) async {
      // ARRANGE: Launch app
      app.main();
      
      // Pump until no more frames (handles splash screens, initial animations)
      await tester.pumpAndSettle(Duration(seconds: 3));
      // Extended duration for real app initialization (API calls, local storage)
      
      // Verify login screen loaded
      expect(find.byKey(Key('login_screen')), findsOneWidget);
      
      // ACT: Fill credentials
      await tester.enterText(
        find.byKey(Key('email_field')), 
        'test@example.com',
      );
      
      await tester.enterText(
        find.byKey(Key('password_field')), 
        'TestPassword123!',
      );
      
      // Hide keyboard (important on real devices)
      await tester.testTextInput.receiveAction(TextInputAction.done);
      await tester.pump();
      
      // Tap login button
      await tester.tap(find.byKey(Key('login_button')));
      
      // Wait for network request and navigation
      await tester.pumpAndSettle(Duration(seconds: 2));
      
      // ASSERT: Verify dashboard loaded
      expect(find.byKey(Key('dashboard_screen')), findsOneWidget);
      expect(find.text('Welcome, test@example.com'), findsOneWidget);
      
      // Verify navigation stack (back button behavior)
      // Should not be able to go back to login
      expect(find.byKey(Key('login_screen')), findsNothing);
    });
    
    testWidgets('user sees error on invalid credentials', 
        (WidgetTester tester) async {
      app.main();
      await tester.pumpAndSettle();
      
      // Enter invalid credentials
      await tester.enterText(
        find.byKey(Key('email_field')), 
        'wrong@example.com',
      );
      await tester.enterText(
        find.byKey(Key('password_field')), 
        'wrongpassword',
      );
      
      await tester.tap(find.byKey(Key('login_button')));
      await tester.pumpAndSettle(Duration(seconds: 2));
      
      // Should still be on login screen
      expect(find.byKey(Key('login_screen')), findsOneWidget);
      
      // Error message displayed
      expect(find.text('Invalid credentials'), findsOneWidget);
    });
  });
}
```

**Explanation:**

- **Keys for stability**: Using `Key('login_screen')` is essential because text might change with localization, but keys remain constant.
- **Extended timeouts**: Real network requests take time. Use `pumpAndSettle(Duration(seconds: 2))` instead of default to wait for API responses.
- **Keyboard handling**: On real devices, the keyboard can obscure buttons. `receiveAction(TextInputAction.done)` dismisses the keyboard before tapping.
- **State verification**: Verify both the presence of expected widgets (dashboard) and absence of previous screens (login), ensuring navigation worked correctly.

### **Handling Async Initialization**

```dart
// File: integration_test/onboarding_test.dart

void main() {
  IntegrationTestWidgetsFlutterBinding.ensureInitialized();
  
  testWidgets('complete onboarding flow for new user', 
      (WidgetTester tester) async {
    // Launch app
    app.main();
    
    // Handle initialization that includes:
    // - Local database setup (SQLite)
    // - Remote config fetching (Firebase)
    // - Authentication state checking (token validation)
    
    // First pump to start app
    await tester.pump();
    
    // Wait for specific condition rather than arbitrary time
    await tester.pumpUntilFound(
      find.byKey(Key('onboarding_page_1')),
      timeout: Duration(seconds: 10),
    );
    // Custom extension method shown below
    
    // Onboarding page 1
    expect(find.text('Welcome to MyApp'), findsOneWidget);
    await tester.tap(find.text('Next'));
    await tester.pumpAndSettle();
    
    // Onboarding page 2
    expect(find.byKey(Key('onboarding_page_2')), findsOneWidget);
    await tester.tap(find.text('Get Started'));
    await tester.pumpAndSettle();
    
    // Should land on home screen
    expect(find.byKey(Key('home_screen')), findsOneWidget);
  });
}

// Extension helper for waiting with timeout
extension PumpUntilFound on WidgetTester {
  Future<void> pumpUntilFound(
    Finder finder, {
    Duration timeout = const Duration(seconds: 10),
  }) async {
    final endTime = DateTime.now().add(timeout);
    
    while (DateTime.now().isBefore(endTime)) {
      await pump();
      
      try {
        expect(finder, findsOneWidget);
        return;
      } catch (_) {
        // Widget not found yet, continue waiting
      }
      
      await Future.delayed(Duration(milliseconds: 100));
    }
    
    throw Exception('Timeout waiting for ${finder.toString()}');
  }
}
```

**Explanation:**

- **`pumpUntilFound`**: Helper extension that polls until a widget appears rather than guessing durations. Handles variable network latency gracefully.
- **Real initialization**: Apps often check for existing auth tokens or download configuration on startup. Tests must wait for this to complete.
- **Polling pattern**: Check condition, pump, wait briefly, repeat. More reliable than `sleep()` or long `pumpAndSettle` delays.

---

## **30.3 Testing Complete User Flows**

Integration tests excel at verifying complex workflows that span multiple screens and involve state persistence.

### **E-Commerce Checkout Flow**

```dart
// File: integration_test/checkout_flow_test.dart

void main() {
  IntegrationTestWidgetsFlutterBinding.ensureInitialized();
  
  group('E2E Checkout Flow', () {
    testWidgets('user can add items to cart and checkout', 
        (WidgetTester tester) async {
      app.main();
      await tester.pumpAndSettle(Duration(seconds: 3));
      
      // Step 1: Browse catalog
      expect(find.byKey(Key('catalog_screen')), findsOneWidget);
      
      // Add first item to cart
      await tester.tap(find.byKey(Key('add_to_cart_0')));
      await tester.pumpAndSettle();
      
      // Add second item
      await tester.tap(find.byKey(Key('add_to_cart_1')));
      await tester.pumpAndSettle();
      
      // Verify cart badge shows "2"
      expect(find.text('2'), findsOneWidget);
      
      // Step 2: Navigate to cart
      await tester.tap(find.byKey(Key('cart_tab')));
      await tester.pumpAndSettle();
      
      expect(find.byKey(Key('cart_screen')), findsOneWidget);
      expect(find.text('Item 1'), findsOneWidget);
      expect(find.text('Item 2'), findsOneWidget);
      
      // Step 3: Proceed to checkout
      await tester.tap(find.byKey(Key('checkout_button')));
      await tester.pumpAndSettle();
      
      // Step 4: Fill shipping details
      expect(find.byKey(Key('shipping_form')), findsOneWidget);
      
      await tester.enterText(
        find.byKey(Key('name_field')), 
        'John Doe',
      );
      await tester.enterText(
        find.byKey(Key('address_field')), 
        '123 Test Street',
      );
      await tester.enterText(
        find.byKey(Key('city_field')), 
        'Test City',
      );
      
      // Scroll to find button (form might be long)
      await tester.drag(
        find.byType(SingleChildScrollView),
        Offset(0, -300),
      );
      await tester.pump();
      
      await tester.tap(find.byKey(Key('continue_payment')));
      await tester.pumpAndSettle(Duration(seconds: 2));
      
      // Step 5: Payment screen
      expect(find.byKey(Key('payment_screen')), findsOneWidget);
      
      // Enter test credit card (using test environment)
      await tester.enterText(
        find.byKey(Key('card_number')), 
        '4242424242424242',
      );
      await tester.enterText(find.byKey(Key('expiry')), '12/25');
      await tester.enterText(find.byKey(Key('cvv')), '123');
      
      await tester.tap(find.byKey(Key('place_order')));
      
      // Wait for payment processing (network call)
      await tester.pumpAndSettle(Duration(seconds: 3));
      
      // Step 6: Order confirmation
      expect(find.byKey(Key('order_confirmation')), findsOneWidget);
      expect(find.text('Order Placed Successfully!'), findsOneWidget);
      
      // Verify order number format (regex matching in test)
      final orderNumberFinder = find.byKey(Key('order_number'));
      final orderNumber = (orderNumberFinder.evaluate().first.widget as Text).data;
      expect(orderNumber, matches(RegExp(r'ORD-\d{6}')));
    });
    
    testWidgets('cart persists across app restarts', 
        (WidgetTester tester) async {
      // First session: Add item to cart
      app.main();
      await tester.pumpAndSettle();
      
      await tester.tap(find.byKey(Key('add_to_cart_0')));
      await tester.pumpAndSettle();
      
      // Simulate app restart by relaunching
      // In real scenario, this would be separate test runs with state
      
      // For demo purposes, navigate away and back
      await tester.tap(find.byKey(Key('profile_tab')));
      await tester.pumpAndSettle();
      
      await tester.tap(find.byKey(Key('cart_tab')));
      await tester.pumpAndSettle();
      
      // Item should still be in cart (persisted in local storage)
      expect(find.text('Item 0'), findsOneWidget);
    });
  });
}
```

**Explanation:**

- **Multi-step workflows**: Tests mirror real user journeys: Browse → Cart → Checkout → Payment → Confirmation. Each step verifies the previous action succeeded.
- **State persistence**: Tests verify that cart state survives navigation changes and (implicitly) app restarts through local storage.
- **Scrolling**: `drag()` on scrollable widgets reveals off-screen content like forms that extend beyond the viewport.
- **Regex validation**: Verifying order number format ensures the backend integration is working correctly, not just that any text appears.
- **Network timing**: Payment processing requires longer `pumpAndSettle` durations (3+ seconds) for real API calls to complete.

### **Navigation and Deep Linking**

```dart
// File: integration_test/navigation_test.dart

void main() {
  IntegrationTestWidgetsFlutterBinding.ensureInitialized();
  
  group('Navigation Flows', () {
    testWidgets('bottom navigation switches tabs correctly', 
        (WidgetTester tester) async {
      app.main();
      await tester.pumpAndSettle();
      
      // Verify initial tab (Home)
      expect(find.byKey(Key('home_tab_active')), findsOneWidget);
      
      // Tap Search tab
      await tester.tap(find.byKey(Key('search_tab')));
      await tester.pumpAndSettle();
      expect(find.byKey(Key('search_screen')), findsOneWidget);
      
      // Tap Profile tab
      await tester.tap(find.byKey(Key('profile_tab')));
      await tester.pumpAndSettle();
      expect(find.byKey(Key('profile_screen')), findsOneWidget);
      
      // Go back to Home
      await tester.tap(find.byKey(Key('home_tab')));
      await tester.pumpAndSettle();
      expect(find.byKey(Key('home_screen')), findsOneWidget);
    });
    
    testWidgets('back button navigation works correctly', 
        (WidgetTester tester) async {
      app.main();
      await tester.pumpAndSettle();
      
      // Navigate to detail screen
      await tester.tap(find.byKey(Key('item_0')));
      await tester.pumpAndSettle();
      
      expect(find.byKey(Key('detail_screen')), findsOneWidget);
      
      // Press back (simulated)
      await tester.pageBack(); // Android-style back
      await tester.pumpAndSettle();
      
      // Should return to list
      expect(find.byKey(Key('home_screen')), findsOneWidget);
      expect(find.byKey(Key('detail_screen')), findsNothing);
    });
    
    testWidgets('modal dialogs and bottom sheets', 
        (WidgetTester tester) async {
      app.main();
      await tester.pumpAndSettle();
      
      // Open settings modal
      await tester.tap(find.byKey(Key('settings_button')));
      await tester.pumpAndSettle();
      
      // Verify modal appears
      expect(find.byKey(Key('settings_modal')), findsOneWidget);
      
      // Close via cancel button
      await tester.tap(find.byKey(Key('cancel_button')));
      await tester.pumpAndSettle();
      
      // Verify modal dismissed
      expect(find.byKey(Key('settings_modal')), findsNothing);
    });
  });
}
```

**Explanation:**

- **Tab navigation**: Verifies that bottom navigation bars correctly switch content while maintaining state of inactive tabs.
- **Page back**: `pageBack()` simulates the Android back button or iOS swipe-back gesture.
- **Modal testing**: Tests verify both opening and proper dismissal of dialogs/bottom sheets, ensuring no zombie widgets remain in tree.

---

## **30.4 Working with Real Devices and Emulators**

Integration tests behave differently on simulators vs. physical devices due to hardware limitations and platform permissions.

### **Device-Specific Testing**

```dart
// File: integration_test/device_specific_test.dart
import 'dart:io' show Platform;

void main() {
  IntegrationTestWidgetsFlutterBinding.ensureInitialized();
  
  group('Device Specific Features', () {
    testWidgets('camera integration on device', (WidgetTester tester) async {
      // Skip if running on simulator (no camera)
      if (Platform.isIOS) {
        // Check if running on simulator
        final isSimulator = await isRunningOnSimulator();
        if (isSimulator) {
          print('Skipping camera test on simulator');
          return;
        }
      }
      
      app.main();
      await tester.pumpAndSettle();
      
      // Navigate to camera screen
      await tester.tap(find.byKey(Key('camera_button')));
      await tester.pumpAndSettle();
      
      // Grant camera permission (must be handled via native automation)
      // This requires additional setup in iOS/Android test runners
      
      // Take photo
      await tester.tap(find.byKey(Key('shutter_button')));
      await tester.pumpAndSettle(Duration(seconds: 2));
      
      // Verify photo preview displayed
      expect(find.byKey(Key('photo_preview')), findsOneWidget);
    });
    
    testWidgets('biometric authentication', (WidgetTester tester) async {
      // Only test on real devices with biometric hardware
      app.main();
      await tester.pumpAndSettle();
      
      await tester.tap(find.byKey(Key('biometric_login')));
      await tester.pumpAndSettle();
      
      // Trigger system biometric prompt
      // This requires mocking or using platform-specific test hooks
      // Actual biometric UI is controlled by OS, not Flutter
    });
    
    testWidgets('push notifications handling', (WidgetTester tester) async {
      app.main();
      await tester.pumpAndSettle();
      
      // Simulate push notification (requires Firebase Test Lab or similar)
      // Verify app handles notification tap correctly
      
      // For local testing, simulate via deep link or method channel
    });
  });
}

// Helper to detect simulator
Future<bool> isRunningOnSimulator() async {
  // Implementation varies by platform
  // iOS: Check for simulator-specific environment variables
  // Android: Check for emulator-specific properties
  return false; // Placeholder
}
```

**Explanation:**

- **Platform detection**: Use `dart:io` `Platform` class to conditionally run tests based on OS (iOS vs Android) and device type (simulator vs physical).
- **Hardware limitations**: Simulators lack cameras, biometric sensors, and GPS accuracy. Tests must skip or mock these on unsupported environments.
- **Permissions**: Camera, microphone, and location permissions require native setup in iOS `Info.plist` and Android `AndroidManifest.xml`, plus pre-authorization in test setup.

### **Performance Metrics Collection**

```dart
// File: integration_test/performance_test.dart

void main() {
  IntegrationTestWidgetsFlutterBinding.ensureInitialized();
  
  group('Performance Tests', () {
    testWidgets('measure scroll performance', (WidgetTester tester) async {
      // Enable performance tracking
      final binding = IntegrationTestWidgetsFlutterBinding.instance;
      
      app.main();
      await tester.pumpAndSettle();
      
      // Start timeline trace
      await binding.traceAction(() async {
        // Perform scroll action
        await tester.fling(
          find.byType(ListView),
          Offset(0, -500),
          1000,
        );
        await tester.pumpAndSettle();
      }, reportKey: 'scroll_performance');
      
      // Results saved to timeline_summary.json
    });
    
    testWidgets('measure frame build times', (WidgetTester tester) async {
      app.main();
      
      // Monitor frame build times
      final stopwatch = Stopwatch()..start();
      
      await tester.pump();
      final firstFrameTime = stopwatch.elapsedMilliseconds;
      
      // Trigger complex build
      await tester.tap(find.byKey(Key('load_data')));
      await tester.pump();
      
      stopwatch.reset();
      await tester.pumpAndSettle();
      final buildTime = stopwatch.elapsedMilliseconds;
      
      print('First frame: ${firstFrameTime}ms');
      print('Data load build: ${buildTime}ms');
      
      // Assert performance standards
      expect(firstFrameTime, lessThan(1000)); // Should render in < 1s
    });
  });
}
```

**Explanation:**

- **`traceAction()`**: Captures detailed timeline data including frame build times, rasterization, and GPU usage during specific actions.
- **Performance budgets**: Tests enforce that operations complete within time limits (e.g., first frame < 1000ms), preventing performance regressions.
- **Timeline data**: Output files (`timeline_summary.json`) can be analyzed in Chrome DevTools or uploaded to CI dashboards.

---

## **30.5 CI/CD Integration**

Automating integration tests ensures quality checks run on every commit before deployment.

### **GitHub Actions Configuration**

```yaml
# File: .github/workflows/integration_tests.yml
name: Integration Tests

on:
  push:
    branches: [main, develop]
  pull_request:
    branches: [main]

jobs:
  ios:
    runs-on: macos-latest
    steps:
      - uses: actions/checkout@v3
      
      - name: Setup Flutter
        uses: subosito/flutter-action@v2
        with:
          flutter-version: '3.16.0'
          channel: 'stable'
      
      - name: Install dependencies
        run: flutter pub get
      
      - name: Start iOS Simulator
        run: |
          xcrun simctl boot "iPhone 14"
          xcrun simctl list devices
      
      - name: Run Integration Tests
        run: |
          flutter test integration_test/ \
            --device-id "iPhone 14" \
            --coverage \
            --reporter expanded
      
      - name: Upload screenshots on failure
        if: failure()
        uses: actions/upload-artifact@v3
        with:
          name: ios-test-screenshots
          path: screenshots/

  android:
    runs-on: macos-latest # macOS required for hardware acceleration
    steps:
      - uses: actions/checkout@v3
      
      - name: Setup Flutter
        uses: subosito/flutter-action@v2
      
      - name: Setup Android SDK
        uses: android-actions/setup-android@v2
      
      - name: Start Android Emulator
        uses: reactivecircus/android-emulator-runner@v2
        with:
          api-level: 33
          arch: x86_64
          profile: pixel_6
          script: |
            flutter test integration_test/ --coverage
      
      - name: Upload results
        if: always()
        uses: actions/upload-artifact@v3
        with:
          name: android-test-results
          path: build/integration_test_results/
```

**Explanation:**

- **Platform-specific jobs**: iOS tests require macOS runners with Xcode; Android tests can run on Linux but macOS provides hardware acceleration for the emulator.
- **Emulator setup**: GitHub Actions uses `reactivecircus/android-emulator-runner` to boot and configure Android emulators with specific API levels and device profiles.
- **Device ID**: `--device-id` specifies which connected device/simulator to use when multiple are available.
- **Artifact upload**: Screenshots and test results are preserved as artifacts when tests fail, enabling debugging without local reproduction.

### **Codemagic Configuration**

```yaml
# File: codemagic.yaml
workflows:
  integration-test-workflow:
    name: Integration Tests
    instance_type: mac_mini_m1
    max_build_duration: 60
    
    environment:
      flutter: stable
      xcode: latest
      cocoapods: default
      
    scripts:
      - name: Get Flutter packages
        script: flutter packages pub get
        
      - name: Run iOS Integration Tests
        script: |
          xcrun simctl list devices
          flutter test integration_test/ \
            --device-id "iPhone 15" \
            --dart-define=ENV=test
            
      - name: Build Android Test APK
        script: |
          flutter build apk --debug
          flutter build apk --debug --target=integration_test/app_test.dart
          
    publishing:
      email:
        recipients:
          - team@example.com
        notify:
          success: false
          failure: true
```

**Explanation:**

- **Instance type**: `mac_mini_m1` provides Apple Silicon for fast iOS builds and emulator performance.
- **Dart defines**: `--dart-define=ENV=test` passes environment variables to the app, allowing test-specific configuration (API endpoints, feature flags).
- **Test APK building**: Android integration tests require building both the app APK and the test APK separately before running.

### **Test Reporting and Flakiness Handling**

```dart
// File: integration_test/helper/test_helper.dart
import 'dart:io';
import 'package:flutter_test/flutter_test.dart';

// Retry mechanism for flaky tests
Future<void> runWithRetries(
  Future<void> Function() test, {
  int maxRetries = 3,
}) async {
  for (var i = 0; i < maxRetries; i++) {
    try {
      await test();
      return; // Success
    } catch (e) {
      if (i == maxRetries - 1) rethrow;
      print('Test failed, retrying... ($i/$maxRetries)');
      await Future.delayed(Duration(seconds: 2));
    }
  }
}

// Screenshot on failure extension
extension ScreenshotOnFailure on WidgetTester {
  Future<void> takeScreenshot(String name) async {
    final binding = IntegrationTestWidgetsFlutterBinding.instance;
    await binding.takeScreenshot(name);
    // Saves to device storage, pulled by CI
  }
}

// Usage in test
void main() {
  IntegrationTestWidgetsFlutterBinding.ensureInitialized();
  
  testWidgets('flaky network test', (WidgetTester tester) async {
    await runWithRetries(() async {
      app.main();
      await tester.pumpAndSettle();
      
      await tester.tap(find.byKey(Key('network_action')));
      await tester.pumpAndSettle(Duration(seconds: 5));
      
      // If this fails, it will retry
      expect(find.byKey(Key('success')), findsOneWidget);
    }, maxRetries: 3);
  });
}
```

**Explanation:**

- **Retry logic**: Network-dependent tests are inherently flaky. Wrapping tests in retry logic reduces false negatives in CI.
- **Screenshots**: `takeScreenshot()` captures the UI state at failure points. Critical for debugging why a test failed on CI when it passes locally.
- **Test isolation**: Each retry should ideally restart the app (`app.main()`) to ensure clean state, though this adds execution time.

---

## **Chapter Summary**

In this chapter, we covered end-to-end testing strategies that ensure your application works correctly in real-world conditions:

### **Key Takeaways:**

1. **Integration Test Setup**: Use `IntegrationTestWidgetsFlutterBinding.ensureInitialized()` to connect to native platforms. Tests reside in `integration_test/` directory, separate from unit tests.

2. **Real App Testing**: Launch actual app with `app.main()` rather than isolated widgets. Tests execute real network calls, database operations, and platform channels.

3. **User Flows**: Test complete workflows (login → dashboard → actions) spanning multiple screens. Verify state persistence across navigation and app restarts.

4. **Device Handling**: Account for hardware differences (simulator vs. physical). Skip tests requiring unavailable hardware (camera, biometrics) on unsupported environments.

5. **Performance Testing**: Use `traceAction()` to capture frame timing and rendering performance. Enforce performance budgets in CI to prevent regressions.

6. **CI/CD Integration**: Configure GitHub Actions, Codemagic, or Bitrise to run tests on emulators/simulators on every PR. Preserve artifacts (screenshots, logs) for debugging failures.

7. **Flakiness Mitigation**: Implement retry logic for network-dependent tests. Use explicit waits (`pumpUntilFound`) rather than arbitrary delays.

### **Next Steps:**

Chapter 31 will begin **Part IX: Performance & Optimization**, covering:
- Flutter's rendering pipeline and the 60fps target
- Performance profiling tools (DevTools, Timeline)
- Build optimization techniques
- Memory management and leak detection

---

**End of Chapter 30**

---

# **Next Chapter: Chapter 31 - Performance Fundamentals**

Chapter 31 will transition from testing to optimization, exploring how to measure, analyze, and improve Flutter application performance for smooth 60/120fps experiences.

